# Inverse PCA

Let's explore the "latent space" of `PCA`.

We can think of `PCA` as a kind of factorization or decomposition of our original images into $2$ values each: one that is a new reduced set of features, called the principal components, and the other is a set of common features extracted from all images.

So, each image $I_i$ can now be represented as a multiplication $I_i = PC_i \cdot V$, where $PC_i$ are the principal components of $I_i$, and $V$ are the common values extract from all images.

Let's look at an example.

## Setup

First we download some libraries and data, and import helpers.

In [ ]:
!wget -q https://github.com/PSAM-5020-2025S-A/5020-utils/raw/main/src/data_utils.py
!wget -q https://github.com/PSAM-5020-2025S-A/5020-utils/raw/main/src/image_utils.py

!wget -qO- https://github.com/PSAM-5020-2025S-A/5020-utils/releases/latest/download/att-faces.tar.gz | tar xz
!wget -qO- https://github.com/PSAM-5020-2025S-A/5020-utils/releases/latest/download/metfaces.tar.gz | tar xz

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import PIL.Image as PImage

from numpy.random import normal as np_normal, randint
from os import listdir, path

from data_utils import PCA, StandardScaler
from image_utils import get_pixels, make_image

## Load Dataset

This is the `AT&T` faces dataset with $400$ images that have been cropped and somewhat aligned.

In [ ]:
# img_width = 92

# lists for keeping track of image pixel lists, subject numeric id and subject label
face_pixels = []

# 40 directories
for l in range(1, 41):
  # 10 images per directory
  for i in range(1, 11):
    mimg = PImage.open(f"./data/image/att-faces/s{l}/{i}.pgm")
    img_width = mimg.size[0]
    face_pixels.append(get_pixels(mimg))

## Start Processing

Get some info about our data, and run `PCA` on all images/rows.

In [ ]:
# Dataset shape
print(len(face_pixels), " X ", len(face_pixels[0]))

# display first image
display(make_image(face_pixels[0], width=img_width))

### PCA and Resulting Transformation

We can see how much information from the original data was kept by our `PCA` transformation, and what common components were extracted.

In [ ]:
# run pca and get first 256 PCs
pca = PCA(n_components=256)
faces_df = pca.fit_transform(face_pixels)

print("explained variance:", pca.explained_variance())

# shape of the common values V
print("common values V", pca.components.shape)

# shape of resulting dataframe
print("Resulting DataFrame:", faces_df.shape[0], "rows and", faces_df.shape[1], "features (principal components)")

The common extracted values are themselves $256$ images.

### Reconstruct

We now have each of our images represented with $256$ values. They've been compressed from $10\text{,}304$ pixels to $256$ components. Since we're only using a portion of the total number of components, we won't be able to fully recuperate the original images, but we should be able to get something pretty close since we're keeping more than $90\%$ of the information from the original images.

Instead of breaking our images down into principal components and common factors, we also have an error $e_i$:

$I_i = PC_i \cdot V + e_i$

But, we can still use $PC_i \cdot V$ to reconstruct our original images.

We can transform the $256$ values back into $10\text{,}304$ pixels using the `inverse_transform()` function of our `PCA` object.

In [ ]:
# Reconstruction: transforms the data from PCA space into pixel space
pca_pixels = pca.inverse_transform(faces_df)

for i in range(4):
  idx = randint(0, len(face_pixels))
  display(make_image(face_pixels[idx], width=img_width))
  display(make_image(pca_pixels.loc[idx], width=img_width))

### Common Factors

These are the common values extracted from the dataset. It's the value $V$ from the equation:

$I_i = PC_i \cdot V + e_i$

All reconstructions are built from linear combinations of these features that are common between all images.

There are $256$ of them, one for each $PC$ component extracted, but let's only look at the first $4$, since those are the most important/common shapes present in our images.

In [ ]:
for comp in pca.components[:4]:
  minVal = comp.min()
  maxVal = comp.max()
  # manually mapping to [0, 255]
  pxs01 = 255 * (comp - minVal) / (maxVal - minVal)
  display(make_image(pxs01, width=img_width))

### Standardize the PC values

We have a new dataset that is made up of $256$ PC features.

The range of values for the $PC$ feature are very different. The first few components of each image tend to have values that are a lot larger than the last components. This is expected and part of the process of making more common features be turned into more important components. The components are ordered by the amount of variance they explain.

If we were to use the PC features to train a classifier, we would keep them unscaled, but we can scale them as part of our current exploratory analysis.

This helps understand the distribution of their values and will also help us pick sensible random values for them later.

In [ ]:
# Look at distributions of un-normalized PC values
faces_df.describe().round(3)

In [ ]:
# scale the PCs
pca_scaler = StandardScaler()
faces_pca_std_df = pca_scaler.fit_transform(faces_df)

display(faces_pca_std_df)
display(faces_pca_std_df.describe().round(3))

### Plot distribution of PCs

This is exploratory.

For each image, we'll plot its first $32$ features.

In the same plot, highlight the values for the first $4$ images in our dataset.

All the red dots are the PC values that are needed to reconstruct the first image, shown in relation to the distribution of all the values of the PCs of all images.

In [ ]:
num_pcs = 32
num_faces = 4
cmap = plt.get_cmap("Set1")

plt.figure(figsize=(8,8))
for i in range(0,num_pcs):
  plt.scatter(faces_pca_std_df[f"PC{i}"], faces_pca_std_df.shape[0] * [i], alpha=0.25, color='#2280fa')


for i in range(num_faces):
  pcs = faces_pca_std_df.iloc[i].values[:num_pcs]
  mcolor = cmap(i/num_faces)
  plt.scatter(pcs, range(num_pcs), c=num_pcs*[mcolor], label=f"Image {i}")

plt.xlabel("Normalized Value")
plt.ylabel("Feature")
plt.legend(loc="lower left")
plt.show()

### Most Average Face

Our simplified math representation of the `PCA` transformation shows that we can get face images $I_i$ by multiplying $256$ PC values by $256$ images (the $V$ components):

$PC_i \cdot V \rightarrow I_i$

So far, the PC values we've been using to reconstruct images were coming from the original dataset, but the $V$ values should have enough information about faces that if we multiply any set of $256$ values by $V$ it should give us a face.

Let's start by looking at what happens if we construct an image from the average value of all of the PCs.

This could be the `DataFrame` we get by computing `mean()` for all PCs in their original ranges: `faces_df.mean()`, but since those features were themselves normalized, they would all have an average value of $0$.

This means we can re-construct the most average face of our dataset by `inverse_transforming()` a list of $256$ zeroes.

In [ ]:
avg_std_df = pd.DataFrame([pca.n_components * [0]], columns=faces_df.columns)
avg_pca_df = pca_scaler.inverse_transform(avg_std_df)

avg_img = pca.inverse_transform(avg_pca_df)
display(make_image(avg_img.loc[0], width=img_width))

### Random Faces

We can also reconstruct faces from $256$ random values.

We could pick them completely random, but let's use normal distributions to make sure we don't go too far from the average value of any particular component.

We'll create a `DataFrame` that has $4$ faces represented in scaled `PCA` space.

We'll then un-scale and un-PCA these values to create $4$ random faces that are very unlikely to be in our original dataset, but that are re-created based on real data features that were extracted in the form of principal components.

In [ ]:
num_ifaces = 4

rand_std_pcs = pd.DataFrame(np_normal(size=[num_ifaces, pca.n_components], scale=0.666), columns=faces_df.columns)
rand_pcs = pca_scaler.inverse_transform(rand_std_pcs)
rand_img = pca.inverse_transform(rand_pcs)

for i in range(num_ifaces):
  display(make_image(rand_img.loc[i], width=img_width))

## Bonus Dataset

The re-constructed random faces look like faces, but the `AT&T` images are not very diverse ($40$ people), and, since part of the original experiment with these faces was to see how robust this method was to minor variations in face positioning, the faces aren't very well aligned.

We can see that very clearly in the reconstructions. It's like there are a bunch ($256$ to be exact) faces being superimposed, but they're not aligned.

Here's a dataset that has $1336$ unique faces that have been better aligned and scaled.

Run this cell and then repeat the process starting from the _Start Processing_ cell above.

In [ ]:
img_width = 256
MET_PATH = f"./data/image/metfaces/{img_width}"

face_pixels = []

for f in sorted([f for f in listdir(MET_PATH) if f.endswith(".jpg")]):
  mimg = PImage.open(path.join(MET_PATH, f)).convert("L")
  face_pixels.append(list(mimg.getdata()))

display(mimg)